In [1]:
import pandas as pd
import json, os

import tensorflow as tf
import numpy as np
import csv
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification #, BertModel, BertTokenizer, TFBertForSequenceClassification
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from collections import OrderedDict
import time
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, \
roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Masking
from tensorflow.keras.layers import Embedding, MaxPool1D
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Bidirectional, BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.initializers import glorot_uniform, RandomUniform, lecun_uniform, Constant
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from collections import OrderedDict
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D, GlobalMaxPool1D
import tensorflow.keras.backend as K

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool1D

from sklearn.metrics import classification_report

# from imblearn.over_sampling import RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler

from sklearn.svm import SVC  
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

import nltk
from nltk.tokenize import word_tokenize
import gensim.downloader
import io
from keras_preprocessing.text import tokenizer_from_json
from gensim.models import Word2Vec


D:\anaconda3\envs\tfgpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Set the seeder to have as stable random operations as possible

In [2]:
seed = 123
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [3]:
embedding_algorithms = ["w2v", "bert", "codebert"]
embedding_algorithm = embedding_algorithms[2]

In [4]:
representations = ["bow", "sequences"]
representation_form = representations[1]

In [5]:
# binary or multi-class
multi = True

In [6]:
train_w2v = False

Read data

In [7]:
if representation_form == "bow":
    data = pd.read_csv('bow_data.csv') # bow
else:
    data = pd.read_csv('sequences_data.csv') # sequences of tokens

In [8]:
data.head()

,Vulnerability,Category,Length
0,"f""str$id""""str$id""""str$id"" ...",sql_injection,9
1,client.listentcp() proxy = proxy(proxy_...,xsrf,8
2,"from django.http import httpresponse, httpresp...",open_redirect,9
3,"def write_preset(conn, queryin, descriptin):\t...",sql_injection,175
4,update_query = self.up...,sql_injection,14


In [9]:
if representation_form == "bow":
    bow_size = 237 # number of columns that stand as features

In [10]:
np.max(data["Length"])

392

In [11]:
if representation_form == "sequences":
    word_counts = data["Vulnerability"].apply(lambda x: len(x.split()))
    max_length = word_counts.max()
    print("Maximum number of words:", max_length)


Maximum number of words: 392


In [12]:
label_frequencies = data['Category'].value_counts()
print("Label Frequencies:\n", label_frequencies)
print("Total samples ", len(data))

Label Frequencies:
 sql_injection            1424
xsrf                      976
command_injection         721
path_disclosure           481
open_redirect             442
remote_code_execution     334
xss                       145
Name: Category, dtype: int64
Total samples  4523


In [13]:
if multi == True:
    n_categories = len(label_frequencies) # 7
    n_categories

Word Embedding

Word2Vec - load pre-trained word2vec embeddings - NL knowledge - static embeddings

In [14]:
if representation_form == "sequences":

    # Download the Punkt tokenizer models if not already downloaded
    nltk.download('punkt')

    tokenized_list = [word_tokenize(sentence) for sentence in data["Vulnerability"].tolist()]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iliaskaloup\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
# # Encode each token using Word2Vec embeddings
# def w2vEncoding(model, tokenized_list):
#     encoded_list = []
#     for sentence_tokens in tokenized_list:
#         encoded_sentence = []
#         for token in sentence_tokens:
#             if token in model:
#                 encoded_token = model[token]
#                 encoded_sentence.append(encoded_token)
#         encoded_list.append(encoded_sentence)
    
#     return encoded_list

In [16]:
if embedding_algorithm == "w2v" and representation_form == "sequences" and train_w2v == False:

    w2v_vectors = gensim.downloader.load('word2vec-google-news-300')
    dim = w2v_vectors.vector_size

#     fileEmb = 'w2v_embeddingsIters.txt'
#     w2v_vectors.save_word2vec_format(fileEmb, binary=False)
    
#     embeddings_index = {}
#     f = open(os.path.join('', fileEmb), encoding="utf-8")
#     for line in f:    
#         values = line.split()
#         word = values[0]
#         coefs = np.asarray(values[1:])
#         embeddings_index[word] = coefs   
#     f.close()
    
    
    tokenizer_obj = Tokenizer()   
    tokenizer_obj.fit_on_texts(tokenized_list)
    
    tokenizer_json = tokenizer_obj.to_json()
    tokenizerFile = 'w2v_tokenizer.json'
    with io.open(tokenizerFile, 'w', encoding='utf-8') as f:
        f.write(json.dumps(tokenizer_json, ensure_ascii=False))
        
    with open(tokenizerFile) as f:
        dataTokenizer = json.load(f)
        tokenizer_obj = tokenizer_from_json(dataTokenizer)
        
    sequences = tokenizer_obj.texts_to_sequences(tokenized_list)
    word_index = tokenizer_obj.word_index
    
    lines_pad = pad_sequences(sequences, padding = 'post', maxlen = max_length)
    
    
    num_words = len(word_index) + 1 # +1 for the unknown-zeros
    embedding_matrix = np.zeros((num_words, dim))
    for word, i in word_index.items():
        if i > num_words:
            continue
        #embedding_vector = embeddings_index.get(word)
        embedding_vector = w2v_vectors[word] if word in w2v_vectors else None
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
  

Use corpus to train word2vec vectors on python source code - PL knowledge - static embeddings

In [17]:
# # this should be executed only in the training set during cross-validation
# from gensim.models import Word2Vec

# w2v_model = Word2Vec(sentences=sequences_data["Vulnerability"], vector_size=300, window=5, min_count=1, workers=4)
# w2v_model.save("python_word2vec.model")
# #w2v_model = Word2Vec.load("python_word2vec.model")
# sequences_data["py_w2v"] = w2vEncoding(w2v_model.wv, tokenized_list)

BERT - load pre-trained bert embeddings - NL knowledge - contextual embeddings

In [18]:
if embedding_algorithm == "bert" and representation_form == "sequences": 
    model_variation = "bert-base-uncased" # "roberta-base"
    tokenizer = AutoTokenizer.from_pretrained(model_variation)
    bert = TFAutoModel.from_pretrained(model_variation)
    
    bert_embeddings = bert.get_input_embeddings()
    embedding_matrix = bert_embeddings.weights[0].numpy()
    num_words = len(embedding_matrix)
    print(num_words)
    dim = len(embedding_matrix[0])
    print(dim)

In [19]:
if embedding_algorithm == "bert" and representation_form == "sequences":
    sentences = data["Vulnerability"].tolist()
    sequences = [tokenizer.encode(sente, truncation=True, add_special_tokens=False, return_tensors="tf").numpy() for sente in sentences] # Tokenize the complete sentences
    
    lines_pad = []
    for seq in sequences:
        lines_pad.append(seq[0])
    
    lines_pad = pad_sequences(lines_pad, padding = 'post', maxlen = 512)

CodeBERT - load pre-trained codebert embeddings - PL knowledge - contextual embeddings

In [20]:
if embedding_algorithm == "codebert" and representation_form == "sequences": 
    model_variation = "microsoft/codebert-base"
    tokenizer = AutoTokenizer.from_pretrained(model_variation, do_lower_case=True)
    codebert = TFAutoModel.from_pretrained(model_variation)
    
    codebert_embeddings = codebert.get_input_embeddings()
    embedding_matrix = codebert_embeddings.weights[0].numpy()
    num_words = len(embedding_matrix)
    print(num_words)
    dim = len(embedding_matrix[0])
    print(dim)

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


50265
768


In [21]:
def padSequences(sequences, max_len):
    lines_pad = []
    for sequence in sequences:
        seq = sequence['input_ids'].numpy()[0]
        if len(seq) < max_len:
            for i in range(len(seq), max_len):
                seq = np.append(seq, 0)
        lines_pad.append(seq)
    return lines_pad


In [22]:
if embedding_algorithm == "codebert" and representation_form == "sequences": 

    sentences = data["Vulnerability"].tolist()
    sequences = [tokenizer(sente, return_tensors="tf", truncation=True, add_special_tokens=False) for sente in sentences]
    
    lines_pad = padSequences(sequences, 512)
    lines_pad = [arr.tolist() for arr in lines_pad]
    lines_pad = np.array(lines_pad)

RNN model, LSTM specifically

In [23]:
def buildLstm(max_len, top_words, dim, seed, embedding_matrix, n_outputs):
    
    model=Sequential()
    
    # LSTM Layer
    model.add(Embedding(input_dim=top_words, output_dim=dim, input_length=None, weights=[embedding_matrix], mask_zero=True, trainable=False))
    model.add(LSTM(100, dropout=0.2, return_sequences=False, stateful=False))
    model.add(Activation('relu'))
    model.add(BatchNormalization(momentum=0.0))
    
    # MLP classifier
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    
    # Output Layer
    model.add(Dense(n_outputs))
    model.add(Activation('softmax'))
    optimizer = optimizers.Adam(learning_rate=0.001)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer)
    
    return model

In [24]:
def buildMLP(n_inputs, n_outputs):
    nIn = n_inputs
    model = Sequential()
    #model.add(Dense(256, input_dim=nIn, kernel_constraint=max_norm(3), bias_constraint=max_norm(3)))#hidden
    model.add(Dense(256, input_dim=nIn))#hidden
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128))#hidden
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(256))#hidden
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(n_outputs)) #binary classification
    model.add(Activation('softmax'))#Output # sigmoid # softmax
    optimizer = optimizers.Adam(learning_rate=0.001) #, epsilon=1e-8, decay=1e-6
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer) # 'binary_crossentropy' # "sparse_categorical_crossentropy"
    return model

In [25]:
# Evaluation functions
def recall_metric(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = (true_positives + K.epsilon()) / (possible_positives + K.epsilon())
        return recall

def precision_metric(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = (true_positives + K.epsilon()) / (predicted_positives + K.epsilon())
        return precision

def f1_metric(y_true, y_pred):

    prec = precision_metric(y_true, y_pred)
    rec = recall_metric(y_true, y_pred)
    f1 = 2*((prec*rec)/(prec+rec+K.epsilon()))
    return f1

def f2_metric(y_true, y_pred):

    prec = precision_metric(y_true, y_pred)
    rec = recall_metric(y_true, y_pred)
    f2 = 5*((prec*rec)/(4*prec+rec+K.epsilon()))
    return f2

Cross-Validation

Multi-class Classification: Categorization of all detected vulnerabilities

In [26]:
if multi == True:
    
    # Convert categories to numerical indexes
    category_numerical_indexes, unique_categories = data["Category"].factorize()

    # Create a dictionary mapping each category to its numerical index
    category_to_index = {category: index for index, category in enumerate(unique_categories)}

    # Update the categories in the DataFrame with their numerical indexes
    data["Category_Index"] = data["Category"].map(category_to_index)


Use MLP model and BoW code representation

In [27]:
def runMLPCrossVal_multi(X, y, seed):
    ############## cross validation
    scores=['accuracy', 'precision', 'recall', 'f1']
    values = [np.array([]) for i in range(0, len(scores))]
    score_dict = OrderedDict(zip(scores, values))
    k=10
    f=0
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    
    nb_epoch = 100
    BS = 64
    print("Training...")
    milli_sec1 = int(round(time.time() * 1000))
    
    for train_index, test_index in kfold.split(X, y):
        f = f + 1
        print('fold number= ',f)
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
#         Y_train = np.array(Y_train)
#         Y_train = Y_train.ravel()
#         Y_test = np.array(Y_test)
#         Y_test = Y_test.ravel()
        
#         #sampling
#         X_res, Y_res = RandomOverSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
#         #X_res, Y_res = RandomUnderSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
    
#         #shuffle dataset
#         X_resampled=pd.DataFrame(X_res)
#         Y_resampled=pd.DataFrame(Y_res)
#         newTrain=X_resampled.assign(Label=Y_resampled.values)
#         newTrain = shuffle(newTrain,random_state=seed)
#         X_train=np.array(newTrain.iloc[:, 0:-1 ])
#         X_train=pd.DataFrame(X_train)
#         Y_train=np.array(newTrain.iloc[:, -1 ])
#         Y_train=pd.DataFrame(Y_train)
        
        myModel = buildMLP(X_train.shape[1], n_categories)
            
        print("model summary\m", myModel.summary())
        csv_logger = CSVLogger('log.csv', append=True, separator=',')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
        history = myModel.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = nb_epoch, batch_size = BS, shuffle=False, verbose=1, callbacks=[csv_logger,es,mc])
        
        #load best model
        #model = load_model('best_model.h5')
        myModel.load_weights("best_model.h5")
        #load best model
        #model = load_model('best_model.h5')
        myModel.load_weights("best_model.h5")
        
        #predictions = myModel.predict_classes(X_test, verbose=0)
        predScores = myModel.predict(X_test)
        predictions = np.argmax(predScores, axis=1)
        
        accuracy=accuracy_score(Y_test, predictions)
        precision=precision_score(Y_test, predictions, average='macro')
        recall=recall_score(Y_test, predictions, average='macro')
        f1=f1_score(Y_test, predictions, average='macro')
        conf_matrix = confusion_matrix(Y_test, predictions)
        print(conf_matrix)
        #tn, fp, fn, tp = confusion_matrix(Y_test, predictions).ravel()
        #acc = ((tp+tn)/(tp+tn+fp+fn))
        print("Accuracy:%.2f%%"%(accuracy*100))
        print("Precision:%.2f%%"%(precision*100))
        print("Recall:%.2f%%"%(recall*100))
        print("F1 score:%.2f%%"%(f1*100))
        print(classification_report(Y_test, predictions))
        del myModel
        score_dict['accuracy'] = np.append(score_dict['accuracy'], accuracy)
        score_dict['precision'] = np.append(score_dict['precision'], precision)
        score_dict['recall'] = np.append(score_dict['recall'], recall)
        score_dict['f1'] = np.append(score_dict['f1'], f1)
        
    milli_sec2 = int(round(time.time() * 1000))
    print("Cross Validation is completed after", milli_sec2-milli_sec1)
    
    print("accuracy: %.2f%% (%.2f%%)" % (score_dict['accuracy'].mean()*100, score_dict['accuracy'].std()*100))
    print("precision: %.2f%% (%.2f%%)" % (score_dict['precision'].mean()*100, score_dict['precision'].std()*100))
    print("recall: %.2f%% (%.2f%%)" % (score_dict['recall'].mean()*100, score_dict['recall'].std()*100))
    print("f1: %.2f%% (%.2f%%)" % (score_dict['f1'].mean()*100, score_dict['f1'].std()*100))


In [28]:
if multi == True and representation_form == "bow":
    runMLPCrossVal_multi(data.iloc[:, 0:bow_size], data["Category_Index"], seed)

Use DL models and sequences of tokens code representation

In [29]:
def runDLCrossVal_multi(X, y, max_len, num_words, dim, seed, embedding_matrix, userModel, multi):
    ############## cross validation
    scores=['accuracy', 'precision', 'recall', 'f1']
    values = [np.array([]) for i in range(0, len(scores))]
    score_dict = OrderedDict(zip(scores, values))
    k=10
    f=0
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    
    nb_epoch = 100
    BS = 64
    print("Training...")
    milli_sec1 = int(round(time.time() * 1000))
    
    for train_index, test_index in kfold.split(X, y):
        f = f + 1
        print('fold number= ',f)
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = y[train_index], y[test_index]
#         Y_train = np.array(Y_train)
#         Y_train = Y_train.ravel()
#         Y_test = np.array(Y_test)
#         Y_test = Y_test.ravel()
        
#         #sampling
#         X_res, Y_res = RandomOverSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
#         #X_res, Y_res = RandomUnderSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
    
#         #shuffle dataset
#         X_resampled=pd.DataFrame(X_res)
#         Y_resampled=pd.DataFrame(Y_res)
#         newTrain=X_resampled.assign(Label=Y_resampled.values)
#         newTrain = shuffle(newTrain,random_state=seed)
#         X_train=np.array(newTrain.iloc[:, 0:-1 ])
#         X_train=pd.DataFrame(X_train)
#         Y_train=np.array(newTrain.iloc[:, -1 ])
#         Y_train=pd.DataFrame(Y_train)
        
        if userModel == "lstm":
            myModel = buildLstm(max_len, num_words, dim, seed, embedding_matrix, n_categories)
        print("model summary\m",myModel.summary())
        csv_logger = CSVLogger('log.csv', append=True, separator=',')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
        history = myModel.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = nb_epoch, batch_size = BS, shuffle=False, verbose=1, callbacks=[csv_logger,es,mc])
        
        #load best model
        #model = load_model('best_model.h5')
        myModel.load_weights("best_model.h5")
        
        scores = myModel.evaluate(X_test, Y_test, verbose=0)
        #predictions = myModel.predict_classes(X_test, verbose=0)
        predScores = myModel.predict(X_test)
        predictions = np.argmax(predScores, axis=1)
        accuracy=accuracy_score(Y_test, predictions)
        precision=precision_score(Y_test, predictions, average='macro')
        recall=recall_score(Y_test, predictions, average='macro')
        f1=f1_score(Y_test, predictions, average='macro')
        conf_matrix = confusion_matrix(Y_test, predictions)
        print(conf_matrix)
        print("Accuracy:%.2f%%"%(accuracy*100))
        print("Precision:%.2f%%"%(precision*100))
        print("Recall:%.2f%%"%(recall*100))
        print("F1 score:%.2f%%"%(f1*100))
        print(classification_report(Y_test, predictions))
        del myModel
        score_dict['accuracy'] = np.append(score_dict['accuracy'], accuracy)
        score_dict['precision'] = np.append(score_dict['precision'], precision)
        score_dict['recall'] = np.append(score_dict['recall'], recall)
        score_dict['f1'] = np.append(score_dict['f1'], f1)
        
    milli_sec2 = int(round(time.time() * 1000))
    print("Cross Validation is completed after", milli_sec2-milli_sec1)
    
    print("accuracy: %.2f%% (%.2f%%)" % (score_dict['accuracy'].mean()*100, score_dict['accuracy'].std()*100))
    print("precision: %.2f%% (%.2f%%)" % (score_dict['precision'].mean()*100, score_dict['precision'].std()*100))
    print("recall: %.2f%% (%.2f%%)" % (score_dict['recall'].mean()*100, score_dict['recall'].std()*100))
    print("f1: %.2f%% (%.2f%%)" % (score_dict['f1'].mean()*100, score_dict['f1'].std()*100))


In [30]:
if multi == True and representation_form == "sequences" and train_w2v == False:
    runDLCrossVal_multi(lines_pad, data["Category_Index"].values, 512, num_words, dim, seed, embedding_matrix, "lstm", multi)

Training...
fold number=  1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 768)         38603520  
_________________________________________________________________
lstm (LSTM)                  (None, 100)               347600    
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dense (Dense)                (None, 256)               25856     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_37 (Dropout)        

Epoch 6/100
64/64 [==============================] - 3s 48ms/step - loss: 0.5972 - val_loss: 0.8782

Epoch 00006: val_loss improved from 0.89396 to 0.87819, saving model to best_model.h5
Epoch 7/100
64/64 [==============================] - 3s 48ms/step - loss: 0.5054 - val_loss: 0.9369

Epoch 00007: val_loss did not improve from 0.87819
Epoch 8/100
64/64 [==============================] - 3s 49ms/step - loss: 0.4464 - val_loss: 0.8836

Epoch 00008: val_loss did not improve from 0.87819
Epoch 9/100
64/64 [==============================] - ETA: 0s - loss: 0.3728- ETA: 2s - loss:  - ET - ETA: 0s - loss: 0. - 3s 49ms/step - loss: 0.3728 - val_loss: 0.9890

Epoch 00009: val_loss did not improve from 0.87819
Epoch 10/100
64/64 [==============================] - 3s 48ms/step - loss: 0.3584 - val_loss: 0.9774

Epoch 00010: val_loss did not improve from 0.87819
Epoch 11/100
64/64 [==============================] - 3s 49ms/step - loss: 0.3151 - val_loss: 0.9571: 0.

Epoch 00011: val_loss did not

64/64 [==============================] - 3s 48ms/step - loss: 0.2499 - val_loss: 0.9638

Epoch 00012: val_loss did not improve from 0.81860
Epoch 13/100
64/64 [==============================] - 3s 48ms/step - loss: 0.2499 - val_loss: 0.9330

Epoch 00013: val_loss did not improve from 0.81860
Epoch 14/100
64/64 [==============================] - 3s 48ms/step - loss: 0.2213 - val_loss: 0.8887

Epoch 00014: val_loss did not improve from 0.81860
Epoch 15/100
64/64 [==============================] - 3s 49ms/step - loss: 0.1850 - val_loss: 0.9557

Epoch 00015: val_loss did not improve from 0.81860
Epoch 16/100
64/64 [==============================] - 3s 48ms/step - loss: 0.1764 - val_loss: 1.0311

Epoch 00016: val_loss did not improve from 0.81860
Epoch 00016: early stopping
[[126   2   2   0   3   5   5]
 [  6  83   0   0   5   1   2]
 [  9   2  22   0   5   1   5]
 [  4   2   0   4   4   1   0]
 [  1   0   2   0  24   4   2]
 [ 12   2   0   0   2  53   3]
 [  6   1   0   0   5   1  35]]
Ac

64/64 [==============================] - 7s 63ms/step - loss: 1.5175 - val_loss: 1.3428

Epoch 00001: val_loss improved from inf to 1.34285, saving model to best_model.h5
Epoch 2/100
64/64 [==============================] - 3s 48ms/step - loss: 1.1556 - val_loss: 1.0512

Epoch 00002: val_loss improved from 1.34285 to 1.05117, saving model to best_model.h5
Epoch 3/100
64/64 [==============================] - 3s 48ms/step - loss: 0.9616 - val_loss: 0.9771

Epoch 00003: val_loss improved from 1.05117 to 0.97713, saving model to best_model.h5
Epoch 4/100
64/64 [==============================] - 3s 48ms/step - loss: 0.7922 - val_loss: 0.8900

Epoch 00004: val_loss improved from 0.97713 to 0.88997, saving model to best_model.h5
Epoch 5/100
64/64 [==============================] - 3s 48ms/step - loss: 0.6853 - val_loss: 0.9431

Epoch 00005: val_loss did not improve from 0.88997
Epoch 6/100
64/64 [==============================] - 3s 48ms/step - loss: 0.5583 - val_loss: 0.8472

Epoch 00006: va

64/64 [==============================] - 3s 48ms/step - loss: 0.2940 - val_loss: 0.9082

Epoch 00012: val_loss did not improve from 0.82131
Epoch 13/100
64/64 [==============================] - 3s 48ms/step - loss: 0.2613 - val_loss: 0.9593

Epoch 00013: val_loss did not improve from 0.82131
Epoch 14/100
64/64 [==============================] - 3s 48ms/step - loss: 0.2442 - val_loss: 0.9644

Epoch 00014: val_loss did not improve from 0.82131
Epoch 15/100
64/64 [==============================] - 3s 48ms/step - loss: 0.1990 - val_loss: 0.9744

Epoch 00015: val_loss did not improve from 0.82131
Epoch 16/100
64/64 [==============================] - 3s 48ms/step - loss: 0.1906 - val_loss: 0.9234 - los

Epoch 00016: val_loss did not improve from 0.82131
Epoch 17/100
64/64 [==============================] - 3s 48ms/step - loss: 0.1677 - val_loss: 1.0094

Epoch 00017: val_loss did not improve from 0.82131
Epoch 18/100
64/64 [==============================] - 3s 48ms/step - loss: 0.1683 - val_l

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 768)         38603520  
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               347600    
_________________________________________________________________
activation_40 (Activation)   (None, 100)               0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 100)               400       
_________________________________________________________________
dense_32 (Dense)             (None, 256)               25856     
_________________________________________________________________
activation_41 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_61 (Dropout)         (None, 256)              

64/64 [==============================] - 6s 62ms/step - loss: 1.5102 - val_loss: 1.2314

Epoch 00001: val_loss improved from inf to 1.23142, saving model to best_model.h5
Epoch 2/100
64/64 [==============================] - 3s 48ms/step - loss: 1.1213 - val_loss: 1.1882

Epoch 00002: val_loss improved from 1.23142 to 1.18816, saving model to best_model.h5
Epoch 3/100
64/64 [==============================] - 3s 48ms/step - loss: 0.9370 - val_loss: 0.9206

Epoch 00003: val_loss improved from 1.18816 to 0.92057, saving model to best_model.h5
Epoch 4/100
64/64 [==============================] - 3s 48ms/step - loss: 0.7751 - val_loss: 0.8604

Epoch 00004: val_loss improved from 0.92057 to 0.86042, saving model to best_model.h5
Epoch 5/100
64/64 [==============================] - 3s 49ms/step - loss: 0.6716 - val_loss: 0.8342

Epoch 00005: val_loss improved from 0.86042 to 0.83420, saving model to best_model.h5
Epoch 6/100
64/64 [==============================] - 3s 49ms/step - loss: 0.5545 

One more option for training your own word2vec vectors

In [31]:
def trainWord2Vec(tokenized_list, max_length, dim):
    # this should be executed only in the training set during cross-validation
    w2v_model = Word2Vec(sentences=tokenized_list, vector_size=dim, window=5, min_count=1, workers=4, epochs=4)
    #w2v_model.save("python_word2vec.model")
    #w2v_model = Word2Vec.load("python_word2vec.model")

    #     fileEmb = 'w2v_embeddingsIters.txt'
    #     w2v_vectors.save_word2vec_format(fileEmb, binary=False)

    #     embeddings_index = {}
    #     f = open(os.path.join('', fileEmb), encoding="utf-8")
    #     for line in f:    
    #         values = line.split()
    #         word = values[0]
    #         coefs = np.asarray(values[1:])
    #         embeddings_index[word] = coefs   
    #     f.close()


    tokenizer_obj = Tokenizer()   
    tokenizer_obj.fit_on_texts(tokenized_list)

    tokenizer_json = tokenizer_obj.to_json()
    tokenizerFile = 'w2v_new_tokenizer.json'
    with io.open(tokenizerFile, 'w', encoding='utf-8') as f:
        f.write(json.dumps(tokenizer_json, ensure_ascii=False))

    with open(tokenizerFile) as f:
        dataTokenizer = json.load(f)
        tokenizer_obj = tokenizer_from_json(dataTokenizer)

    sequences = tokenizer_obj.texts_to_sequences(tokenized_list)
    word_index = tokenizer_obj.word_index

    lines_pad = pad_sequences(sequences, padding = 'post', maxlen = max_length)


    num_words = len(word_index) + 1 # +1 for the unknown-zeros
    embedding_matrix = np.zeros((num_words, dim))
    for word, i in word_index.items():
        if i > num_words:
            continue
        #embedding_vector = embeddings_index.get(word)
        embedding_vector = w2v_model.wv[word] if word in w2v_model.wv else None
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return lines_pad, embedding_matrix, num_words, tokenizerFile


In [32]:
def testWord2Vec(tokenized_list, tokenizer_path, max_length):
    
    with open(tokenizer_path) as f:
        dataTokenizer = json.load(f)
        tokenizer_obj = tokenizer_from_json(dataTokenizer)
        
    sequences = tokenizer_obj.texts_to_sequences(tokenized_list)
    
    lines_pad = pad_sequences(sequences, padding = 'post', maxlen = max_length)
    
    return lines_pad

In [33]:
def runDLCrossValW2v_multi(X, y, max_len, dim, seed):
    ############## cross validation
    scores=['accuracy', 'precision', 'recall', 'f1']
    values = [np.array([]) for i in range(0, len(scores))]
    score_dict = OrderedDict(zip(scores, values))
    k=10
    f=0
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    
    nb_epoch = 100
    BS = 64
    print("Training...")
    milli_sec1 = int(round(time.time() * 1000))
    
    for train_index, test_index in kfold.split(X, y):
        f = f + 1
        print('fold number= ',f)
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = y[train_index], y[test_index]
        
        X_train, embedding_matrix, num_words, tokenizerFile = trainWord2Vec(X_train, max_len, dim)
        
        X_test = testWord2Vec(X_test, tokenizerFile, max_len)
        
#         Y_train = np.array(Y_train)
#         Y_train = Y_train.ravel()
#         Y_test = np.array(Y_test)
#         Y_test = Y_test.ravel()
        
#         #sampling
#         X_res, Y_res = RandomOverSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
#         #X_res, Y_res = RandomUnderSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
    
#         #shuffle dataset
#         X_resampled=pd.DataFrame(X_res)
#         Y_resampled=pd.DataFrame(Y_res)
#         newTrain=X_resampled.assign(Label=Y_resampled.values)
#         newTrain = shuffle(newTrain,random_state=seed)
#         X_train=np.array(newTrain.iloc[:, 0:-1 ])
#         X_train=pd.DataFrame(X_train)
#         Y_train=np.array(newTrain.iloc[:, -1 ])
#         Y_train=pd.DataFrame(Y_train)
        
        if userModel == "lstm":
            myModel = buildLstm(max_len, num_words, dim, seed, embedding_matrix, n_categories)
        print("model summary\m",myModel.summary())
        csv_logger = CSVLogger('log.csv', append=True, separator=',')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
        
        history = myModel.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = nb_epoch, batch_size = BS, shuffle=False, verbose=1, callbacks=[csv_logger,es,mc])
        
        
        #load best model
        #model = load_model('best_model.h5')
        myModel.load_weights("best_model.h5")
        
        scores = myModel.evaluate(X_test, Y_test, verbose=0)
        #predictions = myModel.predict_classes(X_test, verbose=0)
        predScores = myModel.predict(X_test)
        predictions = np.argmax(predScores, axis=1)
        accuracy=accuracy_score(Y_test, predictions)
        precision=precision_score(Y_test, predictions, average='macro')
        recall=recall_score(Y_test, predictions, average='macro')
        f1=f1_score(Y_test, predictions, average='macro')
        conf_matrix = confusion_matrix(Y_test, predictions)
        print(conf_matrix)
        print("Accuracy:%.2f%%"%(accuracy*100))
        print("Precision:%.2f%%"%(precision*100))
        print("Recall:%.2f%%"%(recall*100))
        print("F1 score:%.2f%%"%(f1*100))
        print(classification_report(Y_test, predictions))
        del myModel
        score_dict['accuracy'] = np.append(score_dict['accuracy'], accuracy)
        score_dict['precision'] = np.append(score_dict['precision'], precision)
        score_dict['recall'] = np.append(score_dict['recall'], recall)
        score_dict['f1'] = np.append(score_dict['f1'], f1)
        
    milli_sec2 = int(round(time.time() * 1000))
    print("Cross Validation is completed after", milli_sec2-milli_sec1)
    
    print("accuracy: %.2f%% (%.2f%%)" % (score_dict['accuracy'].mean()*100, score_dict['accuracy'].std()*100))
    print("precision: %.2f%% (%.2f%%)" % (score_dict['precision'].mean()*100, score_dict['precision'].std()*100))
    print("recall: %.2f%% (%.2f%%)" % (score_dict['recall'].mean()*100, score_dict['recall'].std()*100))
    print("f1: %.2f%% (%.2f%%)" % (score_dict['f1'].mean()*100, score_dict['f1'].std()*100))


In [34]:
if train_w2v == True and multi == True and representation_form == "sequences":
    dim = 300
    runDLCrossValW2v_multi(np.array(tokenized_list), data["Category_Index"].values, 512, dim, seed)